In [8]:
using Revise
using Plots, StatsPlots
default(titlefontsize = 10, lw = 1.5)

## Effect of nutrient input on growth and reproduction

In [12]:
BaseParamCollection().deb

DEBBaseParams
  X_emb_int: Float64 19.42
  K_X: Float64 1.0
  Idot_max_rel: Float64 22.9
  Idot_max_rel_emb: Float64 22.9
  kappa: Float64 0.539
  eta_IA: Float64 0.33
  eta_AS: Float64 0.8
  eta_SA: Float64 0.8
  eta_AR: Float64 0.95
  k_M: Float64 0.59
  k_J: Float64 0.0
  H_p: Float64 100.0
  k_D_G: Array{Float64}((2,)) [1.0, 1.0]
  k_D_M: Array{Float64}((2,)) [1.0, 1.0]
  k_D_A: Array{Float64}((2,)) [1.0, 1.0]
  k_D_R: Array{Float64}((2,)) [1.0, 1.0]
  k_D_h: Array{Float64}((2,)) [1.0, 1.0]
  drc_functs_G: Array{Function}((2,))
  drc_functs_M: Array{Function}((2,))
  drc_functs_A: Array{Function}((2,))
  drc_functs_R: Array{Function}((2,))
  drc_functs_h: Array{Function}((2,))
  drc_params_G: Array{Tuple{Vararg{T, N}} where {N, T}}((2,))
  drc_params_M: Array{Tuple{Vararg{T, N}} where {N, T}}((2,))
  drc_params_A: Array{Tuple{Vararg{T, N}} where {N, T}}((2,))
  drc_params_R: Array{Tuple{Vararg{T, N}} where {N, T}}((2,))
  drc_params_h: Array{Tuple{Vararg{T, N}} where {N, T}}((2,))


In [24]:
1200 / 0.05

24000.0

In [28]:
GlobalBaseParams()

GlobalBaseParams
  t_max: Float64 21.0
  Xdot_in: Float64 1200.0
  V_patch: Float64 0.05
  C_W: Array{Float64}((2,)) [0.0, 0.0]
  units: NamedTuple{(:time, :mass, :volume), Tuple{String, String, String}}


In [30]:
600 / 0.05

12000.0

In [29]:
using DEBBase

plt = plot(
    layout = (1,2), leg = false, 
    title = ["Growth" "Reproduction"], xlabel = "t", 
    size = (650,350)
    )

for Xdot_in in [1.] .* 1200.
    out = simulator(
        BaseParamCollection(
            glb = GlobalBaseParams(Xdot_in = Xdot_in), 
            deb = DEBBaseParams(K_X = 12e3))
        )

    @df out plot!(plt, :t, :S, ylabel = "S", subplot = 1) 
    @df out plot!(plt, :t, :R, ylabel = "R", subplot = 2)
end

display(plt)

DomainError: DomainError with -4.372484202495953:
Exponentiation yielding a complex result requires a complex argument.
Replace x^y with (x+0im)^y, Complex(x)^y, or similar.